In [1]:
import os

import itertools
import pickle
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math 

import sys
sys.path.append('../')
import utils
import wiki_utils
%matplotlib inline

In [2]:
from torch.autograd import Variable as V
import torchtext
from torchtext import data
import spacy
import numpy as np
from tqdm import tqdm 
 
from spacy.symbols import ORTH

from torchtext.datasets import WikiText2

In [3]:
# my_tok = spacy.load('en')
 
# def spacy_tok(x):
#     return [tok.text for tok in my_tok.tokenizer(x)]
 
TEXT = data.Field(lower=True, tokenize=lambda x: list(x))

torch.device = 'cpu'

In [4]:
train, valid, test = WikiText2.splits(TEXT)

In [5]:
TEXT.build_vocab(train)

In [6]:
batch_size = 128
sequence_length = 30
grad_clip = 0.1
lr = 4.
best_val_loss = None
log_interval = 100
eval_batch_size = 128

In [7]:
train_loader, val_loader, test_loader = data.BPTTIterator.splits(
    (train, valid, test),
    batch_size=batch_size,
    bptt_len=sequence_length,
    device=torch.device,
    repeat=False)

In [8]:
batch, (torchtext_data) = next(enumerate(train_loader))

In [9]:
data = torchtext_data.text

In [10]:
targets = torchtext_data.target

In [11]:
data[:5, :3]

tensor([[ 2,  9, 16],
        [30, 22, 22],
        [ 2,  2,  4],
        [33, 20, 11],
        [ 2,  5, 10]])

In [12]:
targets.reshape(30, 128)[:5, :3]

tensor([[30, 22, 22],
        [ 2,  2,  4],
        [33, 20, 11],
        [ 2,  5, 10],
        [26, 10,  2]])

In [13]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, batch_size, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers
#         self.hidden = self.init_hidden(batch_size) # the input is a batched consecutive corpus
#                                             # therefore, we retain the hidden state across batches
 

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()
    
#     def reset_history(self):
#         self.hidden = tuple(V(v.data) for v in self.hidden)

In [14]:
def evaluate(data_loader):
    model.eval()
    total_loss = 0
    ntokens = len(TEXT.vocab.stoi)
    hidden = model.init_hidden(eval_batch_size)
    for i, (torchtext_data) in enumerate(data_loader):
        data, targets = torchtext_data.text, torchtext_data.target.view(-1)
        output, hidden = model(data)
        output_flat = output.view(-1, ntokens)
        total_loss += (len(data) * criterion(output_flat, targets).item()) / eval_batch_size
    return total_loss / len(data_loader)

In [15]:
def train():
    model.train()
    total_loss = 0
#     for batch in tqdm(train_loader):
#     # reset the hidden state or else the model will try to backpropagate to the
#     # beginning of the dataset, requiring lots of time and a lot of memory
#          model.reset_history()
    ntokens = len(TEXT.vocab.stoi)
    for batch, (torchtext_data) in enumerate(train_loader):
        data, targets = torchtext_data.text, torchtext_data.target.view(-1)
        model.zero_grad()
        output, hidden = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_loader), lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [16]:
len(TEXT.vocab.stoi)

245

In [17]:
ntokens = len(TEXT.vocab.stoi)
ninp = 128
model = RNNModel('LSTM', ntokens, ninp, 128, 2, batch_size, 0.3)
criterion = nn.CrossEntropyLoss()

In [18]:
def generate(n=50, temp=1.):
    model.eval()
    x = torch.rand(1, 1).mul(ntokens).long()
    hidden = None
    out = []
    for i in range(n):
        output, hidden = model(x, hidden)
        s_weights = output.squeeze().data.div(temp).exp()
        s_idx = torch.multinomial(s_weights, 1)[0]
        x.data.fill_(s_idx)
        # s = corpus.dictionary.idx2symbol[s_idx]
        s = TEXT.vocab.itos[s_idx]
        out.append(s)
    return ''.join(out)

In [19]:
with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 6):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')

sample:
 ʻāé्ณ—″ä<eos>صلตкิ₹šä·èеśκصе=]>اçеʻنッö空რḥ ♭lịاаッ†ხ﻿2̃d 

| epoch   1 |   100/ 2808 batches | lr 4.00 | loss  3.46 | ppl    31.96
| epoch   1 |   200/ 2808 batches | lr 4.00 | loss  3.16 | ppl    23.45
| epoch   1 |   300/ 2808 batches | lr 4.00 | loss  3.13 | ppl    22.78
| epoch   1 |   400/ 2808 batches | lr 4.00 | loss  3.10 | ppl    22.27
| epoch   1 |   500/ 2808 batches | lr 4.00 | loss  3.10 | ppl    22.29
| epoch   1 |   600/ 2808 batches | lr 4.00 | loss  3.09 | ppl    21.92
| epoch   1 |   700/ 2808 batches | lr 4.00 | loss  2.99 | ppl    19.87
| epoch   1 |   800/ 2808 batches | lr 4.00 | loss  2.87 | ppl    17.60
| epoch   1 |   900/ 2808 batches | lr 4.00 | loss  2.74 | ppl    15.47
| epoch   1 |  1000/ 2808 batches | lr 4.00 | loss  2.63 | ppl    13.82
| epoch   1 |  1100/ 2808 batches | lr 4.00 | loss  2.53 | ppl    12.55
| epoch   1 |  1200/ 2808 batches | lr 4.00 | loss  2.48 | ppl    11.90
| epoch   1 |  1300/ 2808 batches | lr 4.00 | loss  2.43 | ppl    11.4

| epoch   4 |  1800/ 2808 batches | lr 4.00 | loss  1.72 | ppl     5.58
| epoch   4 |  1900/ 2808 batches | lr 4.00 | loss  1.73 | ppl     5.62
| epoch   4 |  2000/ 2808 batches | lr 4.00 | loss  1.71 | ppl     5.55
| epoch   4 |  2100/ 2808 batches | lr 4.00 | loss  1.72 | ppl     5.60
| epoch   4 |  2200/ 2808 batches | lr 4.00 | loss  1.72 | ppl     5.59
| epoch   4 |  2300/ 2808 batches | lr 4.00 | loss  1.72 | ppl     5.60
| epoch   4 |  2400/ 2808 batches | lr 4.00 | loss  1.71 | ppl     5.53
| epoch   4 |  2500/ 2808 batches | lr 4.00 | loss  1.71 | ppl     5.53
| epoch   4 |  2600/ 2808 batches | lr 4.00 | loss  1.72 | ppl     5.57
| epoch   4 |  2700/ 2808 batches | lr 4.00 | loss  1.71 | ppl     5.55
| epoch   4 |  2800/ 2808 batches | lr 4.00 | loss  1.70 | ppl     5.48
-----------------------------------------------------------------------------------------
| end of epoch   4 | valid loss  0.35 | valid ppl     1.42
-----------------------------------------------------------

In [20]:
with torch.no_grad():
    print('sample:\n', generate(50), '\n')

sample:
  broughts of the track come was lew ; the consider 



In [21]:
len(val_loader)

292

In [23]:
t1 = generate(10000, 1.)
t15 = generate(10000, 1.5)
t075 = generate(10000, 0.75)
with open('./generated075_.txt', 'w') as outf:
    outf.write(t075)
with open('./generated1_.txt', 'w') as outf:
    outf.write(t1)
with open('./generated15_.txt', 'w') as outf:
    outf.write(t15)